# 包括的銅先物市場分析（詳細解説付き）

## 分析の全体目的
このノートブックでは、銅先物市場の構造、リスク特性、投資機会を多角的に分析します。

### 主要分析テーマ
1. **期間構造分析**: 限月間の価格関係から市場の需給バランスを読み取る
2. **ボラティリティ分析**: リスクの時間変化と限月別特性を把握する
3. **流動性分析**: 取引コストと市場効率性を評価する
4. **パフォーマンス分析**: リスク調整後リターンを定量評価する
5. **相関・共動関係**: 限月間の連動性を分析する
6. **市場レジーム分析**: 市場環境の変化を特定する

In [ ]:
import sys
import os
import pandas as pd
import numpy as np
import pyodbc
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.gridspec import GridSpec
import seaborn as sns
from scipy import stats
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import warnings

# プロジェクトルートをPythonパスに追加
project_root = os.path.dirname(os.path.dirname(os.path.abspath('__file__')))
sys.path.insert(0, project_root)

from config.database_config import get_connection_string

warnings.filterwarnings('ignore')

# スタイル設定
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['font.size'] = 10
plt.rcParams['figure.dpi'] = 120

# カラーパレット
COLORS = {
    'primary': '#1f77b4',
    'secondary': '#ff7f0e', 
    'tertiary': '#2ca02c',
    'quaternary': '#d62728',
    'quinary': '#9467bd'
}

# 日付軸フォーマット関数
def format_date_axis(ax, data_range_days):
    if data_range_days <= 90:
        ax.xaxis.set_major_locator(mdates.WeekdayLocator(interval=2))
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%m/%d'))
    else:
        ax.xaxis.set_major_locator(mdates.MonthLocator())
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=45, ha='right')

print("ライブラリ読み込み完了")

## データ読み込みと前処理

In [ ]:
def load_data(conn, days=180):
    """データ読み込み関数"""
    query = f"""
    SELECT 
        p.TradeDate,
        t.TenorTypeName,
        p.SettlementPrice,
        p.Volume,
        p.OpenInterest,
        CASE 
            WHEN t.TenorTypeName LIKE 'Generic 1st%' THEN 1
            WHEN t.TenorTypeName LIKE 'Generic 2nd%' THEN 2
            WHEN t.TenorTypeName LIKE 'Generic 3rd%' THEN 3
            WHEN t.TenorTypeName LIKE 'Generic 4th%' THEN 4
            WHEN t.TenorTypeName LIKE 'Generic 5th%' THEN 5
            WHEN t.TenorTypeName LIKE 'Generic 6th%' THEN 6
            WHEN t.TenorTypeName LIKE 'Generic 7th%' THEN 7
            WHEN t.TenorTypeName LIKE 'Generic 8th%' THEN 8
            WHEN t.TenorTypeName LIKE 'Generic 9th%' THEN 9
            WHEN t.TenorTypeName LIKE 'Generic 10th%' THEN 10
            WHEN t.TenorTypeName LIKE 'Generic 11th%' THEN 11
            WHEN t.TenorTypeName LIKE 'Generic 12th%' THEN 12
            ELSE NULL
        END as TenorNumber
    FROM T_CommodityPrice p
    INNER JOIN M_Metal m ON p.MetalID = m.MetalID
    INNER JOIN M_TenorType t ON p.TenorTypeID = t.TenorTypeID
    WHERE 
        m.MetalCode = 'COPPER'
        AND p.TradeDate >= DATEADD(day, -{days}, GETDATE())
        AND p.SettlementPrice IS NOT NULL
    ORDER BY p.TradeDate DESC
    """
    
    df = pd.read_sql(query, conn)
    df['TradeDate'] = pd.to_datetime(df['TradeDate'])
    df = df.dropna(subset=['TenorNumber'])
    return df

# データベース接続・データ読み込み
conn = pyodbc.connect(get_connection_string())
df = load_data(conn, days=180)
data_range_days = (df['TradeDate'].max() - df['TradeDate'].min()).days

# ピボットテーブル作成
price_pivot = df.pivot_table(values='SettlementPrice', index='TradeDate', columns='TenorNumber', aggfunc='mean').sort_index()
volume_pivot = df.pivot_table(values='Volume', index='TradeDate', columns='TenorNumber', aggfunc='mean').sort_index()
oi_pivot = df.pivot_table(values='OpenInterest', index='TradeDate', columns='TenorNumber', aggfunc='mean').sort_index()

print(f"データ期間: {df['TradeDate'].min().date()} - {df['TradeDate'].max().date()}")
print(f"データ件数: {len(df):,}件")
print(f"利用可能限月: {sorted(df['TenorNumber'].unique())}")

## 1. 期間構造分析

### 📊 分析目的
- **市場の需給バランス把握**: コンタンゴ（順鞘）とバックワーデーション（逆鞘）の判定
- **投資機会の発見**: 限月間のスプレッド取引機会を特定
- **市場心理の読み取り**: 将来の需給見通しに対する市場の期待を分析

### 🔍 分析手法
1. **カレンダースプレッド**: 異なる限月間の価格差を計算
2. **期間構造の形状分析**: 曲線の傾きと曲率を定量化
3. **時系列での構造変化**: 市場環境変化に伴う期間構造の推移を追跡

### 📈 グラフの見方
- **正の値（＋）**: コンタンゴ（将来価格＞現在価格）→ 需給緩和期待
- **負の値（－）**: バックワーデーション（現在価格＞将来価格）→ 需給逼迫
- **スプレッドの拡大**: 需給の非対称性増大
- **スプレッドの収束**: 需給バランスの安定化

In [ ]:
# 期間構造メトリクス計算
def calculate_term_structure_metrics(price_data):
    metrics = pd.DataFrame(index=price_data.index)
    
    # カレンダースプレッド（価格差）
    if 1 in price_data.columns and 3 in price_data.columns:
        metrics['M1_M3_Spread'] = price_data[3] - price_data[1]
        metrics['M1_M3_Spread_Pct'] = (price_data[3] / price_data[1] - 1) * 100
    
    if 1 in price_data.columns and 6 in price_data.columns:
        metrics['M1_M6_Spread'] = price_data[6] - price_data[1]
        
    if 1 in price_data.columns and 12 in price_data.columns:
        metrics['M1_M12_Spread'] = price_data[12] - price_data[1]
    
    # フォワードカーブの傾き（短期vs長期）
    front_tenors = [col for col in price_data.columns if col <= 3]
    back_tenors = [col for col in price_data.columns if col >= 9]
    
    if front_tenors and back_tenors:
        front_avg = price_data[front_tenors].mean(axis=1)
        back_avg = price_data[back_tenors].mean(axis=1)
        metrics['Curve_Slope'] = back_avg - front_avg
    
    return metrics.dropna(how='all')

ts_metrics = calculate_term_structure_metrics(price_pivot)

# 期間構造分析の可視化
fig = plt.figure(figsize=(18, 12))
gs = GridSpec(3, 2, figure=fig, hspace=0.4, wspace=0.3)

# 1. カレンダースプレッドの時系列
ax1 = fig.add_subplot(gs[0, :])
if 'M1_M3_Spread' in ts_metrics.columns:
    spread_data = ts_metrics['M1_M3_Spread'].dropna()
    
    # コンタンゴ・バックワーデーション領域を色分け
    contango_mask = spread_data > 0
    backwardation_mask = spread_data < 0
    
    ax1.fill_between(spread_data.index, 0, spread_data, 
                    where=contango_mask, color='lightgreen', alpha=0.4, 
                    label='コンタンゴ', interpolate=True)
    ax1.fill_between(spread_data.index, 0, spread_data, 
                    where=backwardation_mask, color='lightcoral', alpha=0.4, 
                    label='バックワーデーション', interpolate=True)
    
    ax1.plot(spread_data.index, spread_data, color='black', linewidth=2)
    
    # 統計情報追加
    contango_pct = (contango_mask.sum() / len(spread_data)) * 100
    ax1.text(0.02, 0.95, f'コンタンゴ期間: {contango_pct:.1f}%', 
            transform=ax1.transAxes, fontsize=11, 
            bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.8))

ax1.axhline(y=0, color='black', linestyle='-', alpha=0.8)
ax1.set_title('カレンダースプレッド分析（M1-M3）', fontsize=16, weight='bold')
ax1.set_ylabel('スプレッド (USD/tonne)', fontsize=12)
ax1.legend()
ax1.grid(True, alpha=0.3)
format_date_axis(ax1, data_range_days)

# 2. 現在の期間構造形状
ax2 = fig.add_subplot(gs[1, 0])
latest_date = price_pivot.index.max()
latest_prices = price_pivot.loc[latest_date].dropna().sort_index()

if len(latest_prices) > 1:
    ax2.plot(latest_prices.index, latest_prices.values, 'o-', 
            color=COLORS['primary'], linewidth=3, markersize=8)
    
    # 価格ラベル追加
    for tenor, price in latest_prices.items():
        ax2.annotate(f'${price:,.0f}', (tenor, price), 
                    textcoords="offset points", xytext=(0,10), 
                    ha='center', fontsize=9, weight='bold')
    
    ax2.set_title(f'現在の期間構造\n{latest_date.strftime("%Y-%m-%d")}', fontsize=14, weight='bold')
    ax2.set_xlabel('限月', fontsize=12)
    ax2.set_ylabel('価格 (USD/tonne)', fontsize=12)
    ax2.set_xticks(latest_prices.index)
    ax2.set_xticklabels([f'M{int(x)}' for x in latest_prices.index])
    ax2.grid(True, alpha=0.3)

# 3. スプレッド分布分析
ax3 = fig.add_subplot(gs[1, 1])
if 'M1_M3_Spread' in ts_metrics.columns:
    spread_data = ts_metrics['M1_M3_Spread'].dropna()
    
    # ヒストグラム
    n, bins, patches = ax3.hist(spread_data, bins=30, density=True, alpha=0.7, 
                               color=COLORS['primary'], edgecolor='black')
    
    # 統計値表示
    mean_spread = spread_data.mean()
    std_spread = spread_data.std()
    
    ax3.axvline(mean_spread, color='red', linestyle='--', linewidth=2, 
               label=f'平均: {mean_spread:.1f}')
    ax3.axvline(0, color='black', linestyle='-', linewidth=1, alpha=0.5)
    
    ax3.set_title('M1-M3スプレッド分布', fontsize=14, weight='bold')
    ax3.set_xlabel('スプレッド (USD/tonne)', fontsize=12)
    ax3.set_ylabel('頻度密度', fontsize=12)
    ax3.legend()
    ax3.grid(True, alpha=0.3)

# 4. 限月別価格推移
ax4 = fig.add_subplot(gs[2, :])
key_tenors = [1, 3, 6, 12]
colors = [COLORS['primary'], COLORS['secondary'], COLORS['tertiary'], COLORS['quaternary']]

for tenor, color in zip(key_tenors, colors):
    if tenor in price_pivot.columns:
        price_series = price_pivot[tenor].dropna()
        ax4.plot(price_series.index, price_series, 
                color=color, linewidth=2, label=f'M{tenor}', alpha=0.8)

ax4.set_title('限月別価格推移', fontsize=16, weight='bold')
ax4.set_ylabel('価格 (USD/tonne)', fontsize=12)
ax4.legend()
ax4.grid(True, alpha=0.3)
format_date_axis(ax4, data_range_days)

plt.suptitle('銅先物期間構造分析', fontsize=18, weight='bold', y=0.98)
plt.tight_layout()
plt.show()

# 期間構造サマリー
if not ts_metrics.empty:
    latest_metrics = ts_metrics.dropna().iloc[-1]
    current_spread = latest_metrics.get('M1_M3_Spread', 0)
    structure_type = 'コンタンゴ' if current_spread > 0 else 'バックワーデーション'
    
    print(f"\n📋 期間構造分析サマリー")
    print(f"現在のM1-M3スプレッド: ${current_spread:.2f}/tonne")
    print(f"市場構造: {structure_type}")
    
    if 'M1_M3_Spread' in ts_metrics.columns:
        avg_spread = ts_metrics['M1_M3_Spread'].mean()
        print(f"平均スプレッド: ${avg_spread:.2f}/tonne")
        print(f"スプレッドボラティリティ: ${ts_metrics['M1_M3_Spread'].std():.2f}/tonne")

## 2. ボラティリティ・リスク分析

### 📊 分析目的
- **リスク定量化**: 各限月の価格変動リスクを数値化
- **ボラティリティ構造**: 限月間のリスク特性差異を把握
- **リスク管理**: VaRによる最大損失額の推定

### 🔍 分析手法
1. **ローリングボラティリティ**: 20日移動ボラティリティで短期リスクを測定
2. **年率換算**: 日次ボラティリティを年率に変換（×√252）
3. **VaR計算**: 95%信頼区間での最大損失を算出
4. **歪度・尖度**: リターン分布の非対称性と裾の厚さを分析

### 📈 グラフの見方・解釈
- **高ボラティリティ期間**: 市場不安定期、大きな価格変動の可能性
- **低ボラティリティ期間**: 市場安定期、小さな価格変動
- **限月別差異**: 短期限月＞長期限月が一般的（流動性効果）
- **VaR値**: 「1日で5%の確率でこの金額以上の損失」を意味

In [ ]:
# ボラティリティ・リスクメトリクス計算
def calculate_volatility_metrics(price_data):
    returns = price_data.pct_change().dropna()
    
    # ローリングボラティリティ（20日）
    vol_20d = returns.rolling(window=20).std() * np.sqrt(252) * 100
    
    # リスクメトリクス
    risk_metrics = {}
    for tenor in returns.columns:
        tenor_returns = returns[tenor].dropna()
        if len(tenor_returns) > 20:
            risk_metrics[f'M{int(tenor)}'] = {
                'Current_Vol': vol_20d[tenor].iloc[-1] if not vol_20d[tenor].dropna().empty else np.nan,
                'Average_Vol': vol_20d[tenor].mean(),
                'Max_Vol': vol_20d[tenor].max(),
                'Min_Vol': vol_20d[tenor].min(),
                'VaR_95': np.percentile(tenor_returns, 5) * 100,
                'VaR_99': np.percentile(tenor_returns, 1) * 100,
                'Skewness': stats.skew(tenor_returns),
                'Kurtosis': stats.kurtosis(tenor_returns)
            }
    
    return vol_20d, pd.DataFrame(risk_metrics).T

vol_data, risk_summary = calculate_volatility_metrics(price_pivot)

# ボラティリティ分析の可視化
fig = plt.figure(figsize=(18, 12))
gs = GridSpec(3, 3, figure=fig, hspace=0.4, wspace=0.3)

# 1. ボラティリティサーフェス（ヒートマップ）
ax1 = fig.add_subplot(gs[0, :])
vol_clean = vol_data.dropna(how='all')
if not vol_clean.empty:
    # データサイズを調整（表示用）
    vol_display = vol_clean.iloc[-60:]  # 最新60日
    
    im = ax1.imshow(vol_display.T.values, aspect='auto', cmap='plasma', interpolation='bilinear')
    
    # 軸設定
    ax1.set_xticks(range(0, len(vol_display), 10))
    ax1.set_xticklabels([vol_display.index[i].strftime('%m/%d') for i in range(0, len(vol_display), 10)])
    ax1.set_yticks(range(len(vol_display.columns)))
    ax1.set_yticklabels([f'M{int(col)}' for col in vol_display.columns])
    
    # カラーバー
    cbar = plt.colorbar(im, ax=ax1, shrink=0.8)
    cbar.set_label('ボラティリティ (%)', rotation=270, labelpad=20)

ax1.set_title('ボラティリティサーフェス（20日ローリング、年率換算）', fontsize=16, weight='bold')
ax1.set_xlabel('取引日', fontsize=12)
ax1.set_ylabel('限月', fontsize=12)

# 2. 限月別ボラティリティ時系列
ax2 = fig.add_subplot(gs[1, :2])
key_tenors = [1, 3, 6, 12]
colors = [COLORS['primary'], COLORS['secondary'], COLORS['tertiary'], COLORS['quaternary']]

for tenor, color in zip(key_tenors, colors):
    if tenor in vol_data.columns:
        vol_series = vol_data[tenor].dropna()
        ax2.plot(vol_series.index, vol_series, 
                color=color, linewidth=2, label=f'M{tenor}', alpha=0.8)

ax2.set_title('限月別ボラティリティ推移', fontsize=14, weight='bold')
ax2.set_ylabel('年率ボラティリティ (%)', fontsize=12)
ax2.legend()
ax2.grid(True, alpha=0.3)
format_date_axis(ax2, data_range_days)

# 3. 現在のボラティリティ構造
ax3 = fig.add_subplot(gs[1, 2])
if not risk_summary.empty:
    current_vols = risk_summary['Current_Vol'].dropna()
    
    bars = ax3.bar(range(len(current_vols)), current_vols.values, 
                  color=[colors[i % len(colors)] for i in range(len(current_vols))], 
                  alpha=0.8)
    
    # 値ラベル追加
    for i, (bar, val) in enumerate(zip(bars, current_vols.values)):
        ax3.text(bar.get_x() + bar.get_width()/2., bar.get_height() + 0.5,
                f'{val:.1f}%', ha='center', va='bottom', fontsize=9, weight='bold')
    
    ax3.set_title('現在のボラティリティ構造', fontsize=14, weight='bold')
    ax3.set_xlabel('限月', fontsize=12)
    ax3.set_ylabel('ボラティリティ (%)', fontsize=12)
    ax3.set_xticks(range(len(current_vols)))
    ax3.set_xticklabels(current_vols.index)
    ax3.grid(True, alpha=0.3, axis='y')

# 4. VaR分析
ax4 = fig.add_subplot(gs[2, 0])
if not risk_summary.empty:
    var_95 = risk_summary['VaR_95'].dropna()
    var_99 = risk_summary['VaR_99'].dropna()
    
    x = np.arange(len(var_95))
    width = 0.35
    
    bars1 = ax4.bar(x - width/2, var_95.values, width, label='VaR 95%', 
                    color=COLORS['secondary'], alpha=0.8)
    bars2 = ax4.bar(x + width/2, var_99.values, width, label='VaR 99%', 
                    color=COLORS['quaternary'], alpha=0.8)
    
    ax4.set_title('バリューアットリスク', fontsize=14, weight='bold')
    ax4.set_xlabel('限月', fontsize=12)
    ax4.set_ylabel('1日VaR (%)', fontsize=12)
    ax4.set_xticks(x)
    ax4.set_xticklabels(var_95.index)
    ax4.legend()
    ax4.grid(True, alpha=0.3, axis='y')

# 5. リターン分布分析（フロント月）
ax5 = fig.add_subplot(gs[2, 1])
if 1 in price_pivot.columns:
    returns_1m = price_pivot[1].pct_change().dropna() * 100
    
    # ヒストグラム
    n, bins, patches = ax5.hist(returns_1m, bins=30, density=True, alpha=0.7, 
                               color=COLORS['primary'], edgecolor='black')
    
    # 正規分布重ね合わせ
    mu, sigma = returns_1m.mean(), returns_1m.std()
    x = np.linspace(returns_1m.min(), returns_1m.max(), 100)
    normal_dist = stats.norm.pdf(x, mu, sigma)
    ax5.plot(x, normal_dist, 'r-', linewidth=2, label='正規分布')
    
    ax5.axvline(np.percentile(returns_1m, 5), color='orange', linestyle='--', 
               alpha=0.8, label=f'VaR 95%: {np.percentile(returns_1m, 5):.2f}%')
    
    ax5.set_title('リターン分布（M1）', fontsize=14, weight='bold')
    ax5.set_xlabel('日次リターン (%)', fontsize=12)
    ax5.set_ylabel('密度', fontsize=12)
    ax5.legend()
    ax5.grid(True, alpha=0.3)

# 6. リスクメトリクステーブル
ax6 = fig.add_subplot(gs[2, 2])
ax6.axis('off')

if not risk_summary.empty:
    # 主要メトリクスを選択
    display_data = risk_summary[['Current_Vol', 'Average_Vol', 'VaR_95']].round(2)
    
    table_data = []
    for idx, row in display_data.iterrows():
        table_data.append([
            idx,
            f"{row['Current_Vol']:.1f}%",
            f"{row['Average_Vol']:.1f}%",
            f"{row['VaR_95']:.2f}%"
        ])
    
    table = ax6.table(cellText=table_data,
                     colLabels=['限月', '現在Vol', '平均Vol', 'VaR95%'],
                     cellLoc='center',
                     loc='center')
    table.auto_set_font_size(False)
    table.set_fontsize(9)
    table.scale(1, 2)
    
    ax6.set_title('リスクメトリクス要約', fontsize=14, weight='bold', pad=20)

plt.suptitle('銅先物ボラティリティ・リスク分析', fontsize=18, weight='bold', y=0.98)
plt.tight_layout()
plt.show()

# リスク分析サマリー
if not risk_summary.empty:
    print(f"\n⚠️ リスク分析サマリー")
    highest_vol = risk_summary['Current_Vol'].idxmax()
    lowest_vol = risk_summary['Current_Vol'].idxmin()
    print(f"最高ボラティリティ: {highest_vol} ({risk_summary.loc[highest_vol, 'Current_Vol']:.1f}%)")
    print(f"最低ボラティリティ: {lowest_vol} ({risk_summary.loc[lowest_vol, 'Current_Vol']:.1f}%)")
    
    if 'M1' in risk_summary.index:
        m1_var = risk_summary.loc['M1', 'VaR_95']
        print(f"フロント月VaR(95%): {m1_var:.2f}% (1日で5%の確率でこの損失率以上)")

## 3. 流動性・マーケットマイクロストラクチャー分析

### 📊 分析目的
- **取引コスト評価**: スプレッドや流動性による取引コストの把握
- **市場効率性分析**: 価格発見機能と情報伝達効率の評価
- **最適な取引タイミング**: 流動性が高く取引コストが低い時期の特定

### 🔍 分析手法
1. **出来高分析**: 取引活動レベルの測定
2. **建玉分析**: ポジション保有状況の把握
3. **流動性集中度**: 特定限月への取引集中度合い
4. **ターンオーバー率**: 出来高/建玉比率による取引活発度

### 📈 グラフの見方・解釈
- **高出来高**: 流動性が高く、狭いスプレッドで取引可能
- **低出来高**: 流動性リスク、ワイドスプレッドの可能性
- **集中度高**: 特定限月に流動性が集中、他限月は取引困難
- **高ターンオーバー**: 活発な取引、短期売買に適している

In [ ]:
# 流動性メトリクス計算
def calculate_liquidity_metrics(volume_data, oi_data):
    metrics = pd.DataFrame(index=volume_data.index)
    
    # 基本メトリクス
    metrics['Total_Volume'] = volume_data.sum(axis=1, skipna=True)
    metrics['Total_OI'] = oi_data.sum(axis=1, skipna=True)
    
    # 流動性集中度（上位3限月の占有率）
    metrics['Volume_Concentration'] = volume_data.apply(
        lambda x: (x.nlargest(3).sum() / x.sum()) if x.sum() > 0 else np.nan, axis=1
    )
    metrics['OI_Concentration'] = oi_data.apply(
        lambda x: (x.nlargest(3).sum() / x.sum()) if x.sum() > 0 else np.nan, axis=1
    )
    
    # ターンオーバー率
    metrics['Turnover_Ratio'] = metrics['Total_Volume'] / metrics['Total_OI']
    metrics['Turnover_Ratio'] = metrics['Turnover_Ratio'].replace([np.inf, -np.inf], np.nan)
    
    # アムイドマーケット（流動性の一様性）
    metrics['Volume_Diversity'] = volume_data.apply(
        lambda x: len(x[x > x.sum() * 0.05]), axis=1  # 全体の5%以上の出来高を持つ限月数
    )
    
    return metrics.dropna(how='all')

liquidity_metrics = calculate_liquidity_metrics(volume_pivot, oi_pivot)

# 流動性分析の可視化
fig = plt.figure(figsize=(18, 12))
gs = GridSpec(3, 3, figure=fig, hspace=0.4, wspace=0.3)

# 1. 総出来高の時系列
ax1 = fig.add_subplot(gs[0, :2])
if 'Total_Volume' in liquidity_metrics.columns:
    total_vol = liquidity_metrics['Total_Volume'].dropna()
    
    # 出来高とトレンド
    ax1.fill_between(total_vol.index, 0, total_vol, alpha=0.3, color=COLORS['primary'])
    ax1.plot(total_vol.index, total_vol, color=COLORS['primary'], linewidth=2)
    
    # 移動平均
    vol_ma20 = total_vol.rolling(20).mean()
    ax1.plot(vol_ma20.index, vol_ma20, color=COLORS['secondary'], 
            linewidth=2, linestyle='--', label='20日移動平均')
    
    # 出来高レベル評価
    vol_mean = total_vol.mean()
    vol_std = total_vol.std()
    ax1.axhline(vol_mean, color='green', linestyle=':', alpha=0.7, label='平均')
    ax1.axhline(vol_mean + vol_std, color='orange', linestyle=':', alpha=0.7, label='高水準')

ax1.set_title('総出来高推移', fontsize=16, weight='bold')
ax1.set_ylabel('出来高 (契約数)', fontsize=12)
ax1.legend()
ax1.grid(True, alpha=0.3)
format_date_axis(ax1, data_range_days)

# 2. 流動性集中度
ax2 = fig.add_subplot(gs[0, 2])
if 'Volume_Concentration' in liquidity_metrics.columns:
    vol_conc = liquidity_metrics['Volume_Concentration'].dropna() * 100
    oi_conc = liquidity_metrics['OI_Concentration'].dropna() * 100
    
    ax2.plot(vol_conc.index, vol_conc, color=COLORS['primary'], 
            linewidth=2, label='出来高集中度')
    ax2.plot(oi_conc.index, oi_conc, color=COLORS['secondary'], 
            linewidth=2, label='建玉集中度')
    
    ax2.set_title('流動性集中度\n(上位3限月)', fontsize=14, weight='bold')
    ax2.set_ylabel('集中度 (%)', fontsize=12)
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    format_date_axis(ax2, data_range_days)

# 3. 限月別出来高ヒートマップ
ax3 = fig.add_subplot(gs[1, :])
volume_clean = volume_pivot.fillna(0)
if not volume_clean.empty:
    # 正規化（各日の総出来高を100%として）
    volume_norm = volume_clean.div(volume_clean.sum(axis=1), axis=0).fillna(0)
    
    # 最新60日のデータを表示
    vol_display = volume_norm.iloc[-60:]
    
    im = ax3.imshow(vol_display.T.values, aspect='auto', cmap='YlOrRd', interpolation='nearest')
    
    # 軸設定
    ax3.set_xticks(range(0, len(vol_display), 10))
    ax3.set_xticklabels([vol_display.index[i].strftime('%m/%d') for i in range(0, len(vol_display), 10)])
    ax3.set_yticks(range(len(vol_display.columns)))
    ax3.set_yticklabels([f'M{int(col)}' for col in vol_display.columns])
    
    # カラーバー
    cbar = plt.colorbar(im, ax=ax3, shrink=0.8)
    cbar.set_label('出来高シェア', rotation=270, labelpad=20)

ax3.set_title('限月別出来高分布の推移', fontsize=16, weight='bold')
ax3.set_xlabel('取引日', fontsize=12)
ax3.set_ylabel('限月', fontsize=12)

# 4. ターンオーバー率
ax4 = fig.add_subplot(gs[2, 0])
if 'Turnover_Ratio' in liquidity_metrics.columns:
    turnover_clean = liquidity_metrics['Turnover_Ratio'].dropna()
    # 外れ値除去（99%ile以下）
    turnover_clean = turnover_clean[turnover_clean <= turnover_clean.quantile(0.99)]
    
    ax4.plot(turnover_clean.index, turnover_clean, 
            color=COLORS['quaternary'], linewidth=2)
    
    # トレンドライン
    if len(turnover_clean) > 10:
        z = np.polyfit(range(len(turnover_clean)), turnover_clean.values, 1)
        p = np.poly1d(z)
        ax4.plot(turnover_clean.index, p(range(len(turnover_clean))), 
                "--", color='red', alpha=0.8, linewidth=2, label='トレンド')
    
    ax4.set_title('ターンオーバー率', fontsize=14, weight='bold')
    ax4.set_ylabel('出来高/建玉比率', fontsize=12)
    ax4.legend()
    ax4.grid(True, alpha=0.3)
    format_date_axis(ax4, data_range_days)

# 5. 現在の市場構造
ax5 = fig.add_subplot(gs[2, 1])
latest_date = df['TradeDate'].max()
latest_data = df[df['TradeDate'] == latest_date]

if not latest_data.empty:
    tenor_summary = latest_data.groupby('TenorNumber').agg({
        'Volume': 'sum',
        'OpenInterest': 'sum'
    }).fillna(0)
    
    tenors = sorted(tenor_summary.index)
    volume_data = tenor_summary.loc[tenors, 'Volume']
    oi_data = tenor_summary.loc[tenors, 'OpenInterest']
    
    x = np.arange(len(tenors))
    width = 0.35
    
    bars1 = ax5.bar(x - width/2, volume_data, width, label='出来高', 
                    color=COLORS['primary'], alpha=0.8)
    
    ax5_twin = ax5.twinx()
    bars2 = ax5_twin.bar(x + width/2, oi_data, width, label='建玉', 
                        color=COLORS['secondary'], alpha=0.8)
    
    ax5.set_xlabel('限月', fontsize=12)
    ax5.set_ylabel('出来高', color=COLORS['primary'], fontsize=12)
    ax5_twin.set_ylabel('建玉', color=COLORS['secondary'], fontsize=12)
    ax5.set_title(f'現在の市場構造\n{latest_date.strftime("%Y-%m-%d")}', fontsize=14, weight='bold')
    
    ax5.set_xticks(x)
    ax5.set_xticklabels([f'M{int(t)}' for t in tenors])
    
    # 凡例
    lines1, labels1 = ax5.get_legend_handles_labels()
    lines2, labels2 = ax5_twin.get_legend_handles_labels()
    ax5.legend(lines1 + lines2, labels1 + labels2, loc='upper right')
    
    ax5.grid(True, alpha=0.3)

# 6. 流動性統計サマリー
ax6 = fig.add_subplot(gs[2, 2])
ax6.axis('off')

if not liquidity_metrics.empty:
    latest_liq = liquidity_metrics.dropna().iloc[-1]
    avg_liq = liquidity_metrics.mean()
    
    summary_data = [
        ['総出来高', f"{latest_liq.get('Total_Volume', 0):,.0f}", f"{avg_liq.get('Total_Volume', 0):,.0f}"],
        ['総建玉', f"{latest_liq.get('Total_OI', 0):,.0f}", f"{avg_liq.get('Total_OI', 0):,.0f}"],
        ['出来高集中度', f"{latest_liq.get('Volume_Concentration', 0)*100:.1f}%", f"{avg_liq.get('Volume_Concentration', 0)*100:.1f}%"],
        ['ターンオーバー', f"{latest_liq.get('Turnover_Ratio', 0):.2f}", f"{avg_liq.get('Turnover_Ratio', 0):.2f}"]
    ]
    
    table = ax6.table(cellText=summary_data,
                     colLabels=['指標', '現在', '平均'],
                     cellLoc='center',
                     loc='center')
    table.auto_set_font_size(False)
    table.set_fontsize(9)
    table.scale(1, 2.5)
    
    ax6.set_title('流動性統計サマリー', fontsize=14, weight='bold', pad=20)

plt.suptitle('銅先物流動性・マーケットマイクロストラクチャー分析', fontsize=18, weight='bold', y=0.98)
plt.tight_layout()
plt.show()

# 流動性分析サマリー
if not liquidity_metrics.empty:
    latest = liquidity_metrics.dropna().iloc[-1]
    print(f"\n💹 流動性分析サマリー")
    print(f"現在の総出来高: {latest.get('Total_Volume', 0):,.0f} 契約")
    print(f"現在の総建玉: {latest.get('Total_OI', 0):,.0f} 契約")
    print(f"出来高集中度: {latest.get('Volume_Concentration', 0)*100:.1f}% (上位3限月)")
    print(f"ターンオーバー率: {latest.get('Turnover_Ratio', 0):.2f}")
    
    # 流動性評価
    avg_volume = liquidity_metrics['Total_Volume'].mean()
    current_volume = latest.get('Total_Volume', 0)
    liquidity_level = "高" if current_volume > avg_volume * 1.2 else "低" if current_volume < avg_volume * 0.8 else "普通"
    print(f"現在の流動性レベル: {liquidity_level}")

## 4. パフォーマンス・効率性分析

### 📊 分析目的
- **投資効率性評価**: リスク調整後リターンの測定
- **限月間比較**: どの限月が最も効率的な投資対象かを判定
- **ドローダウン分析**: 最大損失期間と回復力の評価

### 🔍 分析手法
1. **シャープレシオ**: 超過リターン/リスクで効率性を測定
2. **累積リターン**: 期間を通じた総リターンを計算
3. **最大ドローダウン**: ピークからの最大下落率を算出
4. **情報比率**: ベンチマーク対比の超過リターンとトラッキングエラー

### 📈 グラフの見方・解釈
- **シャープレシオ＞1.0**: 優秀なリスク調整後リターン
- **累積リターン上昇**: 投資期間を通じて利益獲得
- **小さなドローダウン**: 安定した価格推移、精神的負担軽減
- **高い情報比率**: ベンチマークを上回る安定した超過リターン

In [ ]:
# パフォーマンスメトリクス計算
def calculate_performance_metrics(price_data, benchmark_tenor=1):
    returns = price_data.pct_change().dropna()
    
    metrics = {}
    benchmark_returns = returns[benchmark_tenor] if benchmark_tenor in returns.columns else None
    
    for tenor in price_data.columns:
        price_series = price_data[tenor].dropna()
        tenor_returns = returns[tenor].dropna()
        
        if len(price_series) > 20 and len(tenor_returns) > 20:
            # 基本統計
            total_return = (price_series.iloc[-1] / price_series.iloc[0] - 1) * 100
            ann_return = tenor_returns.mean() * 252 * 100
            ann_vol = tenor_returns.std() * np.sqrt(252) * 100
            
            # リスク調整指標
            sharpe_ratio = (ann_return / ann_vol) if ann_vol > 0 else 0
            
            # ドローダウン計算
            cumulative = (1 + tenor_returns).cumprod()
            running_max = cumulative.expanding().max()
            drawdown = (cumulative - running_max) / running_max * 100
            max_drawdown = drawdown.min()
            
            # 情報比率（ベンチマーク対比）
            info_ratio = np.nan
            if benchmark_returns is not None and tenor != benchmark_tenor:
                excess_returns = tenor_returns - benchmark_returns
                tracking_error = excess_returns.std() * np.sqrt(252) * 100
                if tracking_error > 0:
                    info_ratio = (excess_returns.mean() * 252 * 100) / tracking_error
            
            # 勝率（正のリターンの日数比率）
            win_rate = (tenor_returns > 0).sum() / len(tenor_returns) * 100
            
            metrics[f'M{int(tenor)}'] = {
                'Total_Return': total_return,
                'Annualized_Return': ann_return,
                'Annualized_Vol': ann_vol,
                'Sharpe_Ratio': sharpe_ratio,
                'Max_Drawdown': max_drawdown,
                'Information_Ratio': info_ratio,
                'Win_Rate': win_rate,
                'Calmar_Ratio': ann_return / abs(max_drawdown) if max_drawdown < 0 else 0
            }
    
    return pd.DataFrame(metrics).T

performance_metrics = calculate_performance_metrics(price_pivot)

# パフォーマンス分析の可視化
fig = plt.figure(figsize=(18, 12))
gs = GridSpec(3, 3, figure=fig, hspace=0.4, wspace=0.3)

# 1. 累積リターン比較
ax1 = fig.add_subplot(gs[0, :])
key_tenors = [1, 3, 6, 12]
colors = [COLORS['primary'], COLORS['secondary'], COLORS['tertiary'], COLORS['quaternary']]

for tenor, color in zip(key_tenors, colors):
    if tenor in price_pivot.columns:
        price_series = price_pivot[tenor].dropna()
        if len(price_series) > 1:
            cum_returns = (price_series / price_series.iloc[0] - 1) * 100
            ax1.plot(cum_returns.index, cum_returns, 
                    color=color, linewidth=3, label=f'M{tenor}', alpha=0.8)

ax1.axhline(y=0, color='black', linestyle='--', alpha=0.5)
ax1.set_title('限月別累積リターン比較', fontsize=16, weight='bold')
ax1.set_ylabel('累積リターン (%)', fontsize=12)
ax1.legend()
ax1.grid(True, alpha=0.3)
format_date_axis(ax1, data_range_days)

# 2. リスク・リターン散布図
ax2 = fig.add_subplot(gs[1, 0])
if not performance_metrics.empty:
    scatter = ax2.scatter(performance_metrics['Annualized_Vol'], 
                         performance_metrics['Annualized_Return'],
                         c=performance_metrics['Sharpe_Ratio'], 
                         cmap='RdYlGn', s=120, alpha=0.8, edgecolors='black', linewidth=2)
    
    # ラベル追加
    for idx, row in performance_metrics.iterrows():
        ax2.annotate(idx, (row['Annualized_Vol'], row['Annualized_Return']),
                    xytext=(5, 5), textcoords='offset points', fontsize=10, weight='bold')
    
    plt.colorbar(scatter, ax=ax2, label='シャープレシオ')
    ax2.set_xlabel('年率ボラティリティ (%)')
    ax2.set_ylabel('年率リターン (%)')
    ax2.set_title('リスク・リターン分析', fontsize=14, weight='bold')
    ax2.grid(True, alpha=0.3)

# 3. シャープレシオ比較
ax3 = fig.add_subplot(gs[1, 1])
if not performance_metrics.empty:
    sharpe_data = performance_metrics['Sharpe_Ratio'].dropna().sort_values(ascending=True)
    
    bars = ax3.barh(range(len(sharpe_data)), sharpe_data.values,
                   color=[COLORS['tertiary'] if x > 1 else COLORS['secondary'] if x > 0.5 else COLORS['quaternary'] 
                         for x in sharpe_data.values])
    
    ax3.axvline(x=1.0, color='green', linestyle='--', alpha=0.7, label='優秀ライン')
    ax3.axvline(x=0.5, color='orange', linestyle=':', alpha=0.7, label='及第ライン')
    
    ax3.set_yticks(range(len(sharpe_data)))
    ax3.set_yticklabels(sharpe_data.index)
    ax3.set_xlabel('シャープレシオ')
    ax3.set_title('シャープレシオ比較', fontsize=14, weight='bold')
    ax3.legend()
    ax3.grid(True, alpha=0.3, axis='x')
    
    # 値ラベル
    for i, (bar, val) in enumerate(zip(bars, sharpe_data.values)):
        ax3.text(val + 0.02, bar.get_y() + bar.get_height()/2, 
                f'{val:.2f}', ha='left', va='center', fontsize=9, weight='bold')

# 4. 最大ドローダウン分析
ax4 = fig.add_subplot(gs[1, 2])
if not performance_metrics.empty:
    dd_data = performance_metrics['Max_Drawdown'].dropna().sort_values()
    
    bars = ax4.barh(range(len(dd_data)), dd_data.values,
                   color=[COLORS['tertiary'] if x > -10 else COLORS['secondary'] if x > -20 else COLORS['quaternary'] 
                         for x in dd_data.values])
    
    ax4.set_yticks(range(len(dd_data)))
    ax4.set_yticklabels(dd_data.index)
    ax4.set_xlabel('最大ドローダウン (%)')
    ax4.set_title('最大ドローダウン比較', fontsize=14, weight='bold')
    ax4.grid(True, alpha=0.3, axis='x')
    
    # 値ラベル
    for i, (bar, val) in enumerate(zip(bars, dd_data.values)):
        ax4.text(val - 0.5, bar.get_y() + bar.get_height()/2, 
                f'{val:.1f}%', ha='right', va='center', fontsize=9, weight='bold')

# 5. ドローダウン時系列（フロント月）
ax5 = fig.add_subplot(gs[2, :2])
if 1 in price_pivot.columns:
    price_series = price_pivot[1].dropna()
    returns = price_series.pct_change().dropna()
    cumulative = (1 + returns).cumprod()
    running_max = cumulative.expanding().max()
    drawdown = (cumulative - running_max) / running_max * 100
    
    ax5.fill_between(drawdown.index, 0, drawdown, 
                    alpha=0.5, color=COLORS['quaternary'], label='ドローダウン')
    ax5.plot(drawdown.index, drawdown, 
            color=COLORS['quaternary'], linewidth=2)
    
    # 最大ドローダウン期間をハイライト
    max_dd_idx = drawdown.idxmin()
    ax5.axvline(max_dd_idx, color='red', linestyle='--', alpha=0.8, 
               label=f'最大DD: {drawdown.min():.1f}%')
    
    ax5.axhline(y=0, color='black', linestyle='-', alpha=0.8)
    ax5.set_title('ドローダウン推移（フロント月）', fontsize=14, weight='bold')
    ax5.set_ylabel('ドローダウン (%)', fontsize=12)
    ax5.legend()
    ax5.grid(True, alpha=0.3)
    format_date_axis(ax5, data_range_days)

# 6. パフォーマンス統合表
ax6 = fig.add_subplot(gs[2, 2])
ax6.axis('off')

if not performance_metrics.empty:
    # 主要指標を選択
    display_cols = ['Total_Return', 'Sharpe_Ratio', 'Max_Drawdown', 'Win_Rate']
    display_data = performance_metrics[display_cols].round(2)
    
    table_data = []
    for idx, row in display_data.iterrows():
        table_data.append([
            idx,
            f"{row['Total_Return']:.1f}%",
            f"{row['Sharpe_Ratio']:.2f}",
            f"{row['Max_Drawdown']:.1f}%",
            f"{row['Win_Rate']:.1f}%"
        ])
    
    table = ax6.table(cellText=table_data,
                     colLabels=['限月', 'トータル\nリターン', 'シャープ\nレシオ', '最大\nDD', '勝率'],
                     cellLoc='center',
                     loc='center')
    table.auto_set_font_size(False)
    table.set_fontsize(8)
    table.scale(1, 2.5)
    
    # テーブルスタイル
    for i in range(len(table_data) + 1):
        for j in range(5):
            cell = table[(i, j)]
            if i == 0:  # ヘッダー
                cell.set_facecolor('#34495e')
                cell.set_text_props(weight='bold', color='white')
            else:
                cell.set_facecolor('#ecf0f1' if i % 2 == 0 else 'white')
    
    ax6.set_title('パフォーマンス統合サマリー', fontsize=14, weight='bold', pad=20)

plt.suptitle('銅先物パフォーマンス・効率性分析', fontsize=18, weight='bold', y=0.98)
plt.tight_layout()
plt.show()

# パフォーマンス分析サマリー
if not performance_metrics.empty:
    best_sharpe = performance_metrics['Sharpe_Ratio'].idxmax()
    best_return = performance_metrics['Total_Return'].idxmax()
    lowest_dd = performance_metrics['Max_Drawdown'].idxmax()  # 最も小さな（ゼロに近い）ドローダウン
    
    print(f"\n🏆 パフォーマンス分析サマリー")
    print(f"最高シャープレシオ: {best_sharpe} ({performance_metrics.loc[best_sharpe, 'Sharpe_Ratio']:.2f})")
    print(f"最高リターン: {best_return} ({performance_metrics.loc[best_return, 'Total_Return']:.1f}%)")
    print(f"最小ドローダウン: {lowest_dd} ({performance_metrics.loc[lowest_dd, 'Max_Drawdown']:.1f}%)")
    
    # 投資推奨
    excellent_performers = performance_metrics[performance_metrics['Sharpe_Ratio'] > 1.0]
    if not excellent_performers.empty:
        print(f"\n💡 投資推奨: シャープレシオ>1.0の限月: {', '.join(excellent_performers.index)}")
    else:
        print(f"\n💡 注意: 現在シャープレシオ>1.0の限月はありません")

# データベース接続を閉じる
conn.close()
print("\n✅ 分析完了。データベース接続を閉じました。")

## 📊 総合分析サマリー

### 🎯 分析結果の活用方法

1. **トレーディング戦略**
   - **期間構造**: コンタンゴ時は売り、バックワーデーション時は買いを検討
   - **ボラティリティ**: 高ボラ時は小さなポジション、低ボラ時は大きなポジション
   - **流動性**: 高流動性時に大口取引、低流動性時は小口分割

2. **リスク管理**
   - **VaR**: 日次リスク限度額の設定
   - **ドローダウン**: 最大許容損失の目安
   - **相関**: ポートフォリオ分散効果の評価

3. **投資判断**
   - **シャープレシオ**: リスク調整後の魅力度比較
   - **情報比率**: ベンチマーク対比の超過収益性
   - **流動性コスト**: 取引コストの事前評価

### ⚠️ 分析の限界・注意点

- **過去データベース**: 将来のパフォーマンスを保証するものではない
- **市場環境変化**: マクロ経済環境の変化で関係性が変わる可能性
- **流動性リスク**: 緊急時には通常の流動性が失われる可能性
- **モデルリスク**: 統計モデルの前提条件が崩れるリスク

### 📈 継続的モニタリング項目

1. **期間構造の変化**: 市場の需給バランス変化の早期発見
2. **ボラティリティレジーム**: リスク環境の変化への対応
3. **流動性の変化**: 取引コスト増加の兆候察知
4. **相関の変化**: ポートフォリオ効果の変化監視

---
*このノートブックは銅先物市場の包括的分析フレームワークを提供します。定期的な更新により、最新の市場状況を反映した投資判断が可能になります。*